In [1]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.optimizers import SGD, Adadelta
from keras.layers import Input, Flatten, Dense, Dropout
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import keras
from keras import backend as K


cifar10 = tf.keras.datasets.cifar10
epoch_range = 500

Using TensorFlow backend.


In [2]:
def compressed_weights(model):
    model_weights = np.array(model.get_weights())
    #print(DNN_weights)
    comp = []
    for i in range(len(model_weights)):
        model_weights[i] = model_weights[i].reshape(len(model_weights[i]),-1)
        model_weights[i] = model_weights[i].flatten()
        model_weights[i] = model_weights[i].reshape(len(model_weights[i]),-1)
        #print(model_weights[i].shape)
        for j in range(len(model_weights[i])):
            comp.append(model_weights[i][j])
    comp = np.array(comp).flatten()
    return comp
    

In [3]:
def get_gradients_norm(model, inputs, outputs):
    #from https://stackoverflow.com/questions/51140950/how-to-obtain-the-gradients-in-keras
    """ Gets gradient of model for given inputs and outputs for all weights"""
    grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    grad_sum = 0
    for i in range(len(output_grad)):
        grad_sum += np.sum(output_grad[i])**2
    grad_norm = grad_sum ** 0.5
    return grad_norm


In [4]:
class MY_DNN2(object):
    def __init__(self, width = 32, height = 32, channels = 3):
        self.width = width
        self.height = height
        self.channels = channels
        
        self.shape = (self.width, self.height, self.channels)
        self.optimizer = SGD(lr=0.01, momentum=0.9, decay=1e-8, nesterov=False)
        
        self.model = self.__model()
        self.model.compile(loss='categorical_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])
        
    def __model(self):
        inputs = Input(shape = self.shape)
        x = Flatten(input_shape = self.shape)(inputs)
        x = Dense((self.width * self.height * self.channels), activation='relu')(x)
        x = Dense(400)(x)
        x = Dense(850)(x)
        y = Dropout(0.2)(x)
        outputs = Dense(10,activation='softmax')(y)
        model = Model(inputs=inputs, outputs=outputs)
        model.summary()
        
        return model
        
    def train(self, x_train, y_train, epochs = 500, batch = 256, collect_interval = 3):
        loss = []
        accuracy = []
        weights = []
        grads = []
        
        for cnt in range(epochs):
            history = self.model.fit(x_train, y_train, batch_size=batch, verbose = 0)
            print("epoch:%d ,loss:%s, accuracy:%s "%(cnt, history.history['loss'], history.history['acc']))
            loss.append(history.history['loss'])
            accuracy.append(history.history['acc'])
            grads.append(get_gradients_norm(self.model, x_train, y_train))
            
            if(cnt%collect_interval == 0):
                w = compressed_weights(self.model)
                weights.append(w)

            
        self.loss = loss
        self.accuracy = accuracy
        self.weights = weights
        self.grads = grads
    
        

In [5]:
if __name__ == '__main__':
    if tf.test.gpu_device_name():
        print('GPU found')
    else:
        print("No GPU found")
    
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    #normalization for not getting to big output for 'relu' and other activation
    x_train = x_train/255
    x_test = x_test/255
    y_train = np_utils.to_categorical(y_train, num_classes=10)
    y_test = np_utils.to_categorical(y_test, num_classes=10)
 
    #for dense layer, the parameters = (input_shape + 1) * size;
    
    

GPU found


In [6]:
    DNN2 = MY_DNN2()
    DNN2.train(x_train, y_train)
    DNN2.model.save("CIFAR10_DNN2.h5")
    np.save('CIFAR10_DNN2_LOSS', DNN2.loss)
    np.save('CIFAR10_DNN2_ACC', DNN2.accuracy)
    np.save('CIFAR10_DNN2_WEIGHTS',DNN2.weights)
    np.save('CIFAR10_DNN2_GRADS',DNN2.grads)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3072)              9440256   
_________________________________________________________________
dense_2 (Dense)              (None, 400)               1229200   
_________________________________________________________________
dense_3 (Dense)              (None, 850)               340850    
_________________________________________________________________
dropout_1 (Dropout)          (None, 850)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                8510      
Total para

epoch:108 ,loss:[0.37724988344192506], accuracy:[0.8665599999618531] 
epoch:109 ,loss:[0.3723158095550537], accuracy:[0.8658] 
epoch:110 ,loss:[0.37793200775146485], accuracy:[0.86384] 
epoch:111 ,loss:[0.36297957553863525], accuracy:[0.871640000038147] 
epoch:112 ,loss:[0.35806741819381716], accuracy:[0.872040000038147] 
epoch:113 ,loss:[0.36297060800552366], accuracy:[0.8700200000190735] 
epoch:114 ,loss:[0.3416621067094803], accuracy:[0.87822] 
epoch:115 ,loss:[0.34088800621032717], accuracy:[0.877499999961853] 
epoch:116 ,loss:[0.33500503138542176], accuracy:[0.882079999961853] 
epoch:117 ,loss:[0.326155328540802], accuracy:[0.8843199999618531] 
epoch:118 ,loss:[0.36787842977523805], accuracy:[0.8684599999618531] 
epoch:119 ,loss:[0.3265287825059891], accuracy:[0.885420000038147] 
epoch:120 ,loss:[0.3153807840538025], accuracy:[0.8870400000190735] 
epoch:121 ,loss:[0.3121308500289917], accuracy:[0.887839999961853] 
epoch:122 ,loss:[0.32527321440696716], accuracy:[0.884920000038147]

epoch:231 ,loss:[0.16348108173370363], accuracy:[0.9443400000190735] 
epoch:232 ,loss:[0.15502792932748793], accuracy:[0.947139999961853] 
epoch:233 ,loss:[0.11847008631229401], accuracy:[0.959119999961853] 
epoch:234 ,loss:[0.13676101685523986], accuracy:[0.9526600000190735] 
epoch:235 ,loss:[0.15593149328231812], accuracy:[0.94652] 
epoch:236 ,loss:[0.14418148513317108], accuracy:[0.9508000000190735] 
epoch:237 ,loss:[0.11069013022661209], accuracy:[0.962520000038147] 
epoch:238 ,loss:[0.09874463539123535], accuracy:[0.965420000038147] 
epoch:239 ,loss:[0.07170668168783188], accuracy:[0.976000000038147] 
epoch:240 ,loss:[0.06238532018899918], accuracy:[0.978719999961853] 
epoch:241 ,loss:[0.08344531219959259], accuracy:[0.9710400000381469] 
epoch:242 ,loss:[0.09617150526523591], accuracy:[0.9672399999618531] 
epoch:243 ,loss:[0.10650537006855011], accuracy:[0.9635199999809265] 
epoch:244 ,loss:[0.09850445291996002], accuracy:[0.966119999961853] 
epoch:245 ,loss:[0.12300169862031937],

epoch:353 ,loss:[0.14042901287555695], accuracy:[0.955220000038147] 
epoch:354 ,loss:[0.06082072392731905], accuracy:[0.98006] 
epoch:355 ,loss:[0.026650959069728853], accuracy:[0.9917600000190735] 
epoch:356 ,loss:[0.042322254079580304], accuracy:[0.98672] 
epoch:357 ,loss:[0.05353583365082741], accuracy:[0.9828000000190735] 
epoch:358 ,loss:[0.04423123780786991], accuracy:[0.9850800000190735] 
epoch:359 ,loss:[0.026548242664337158], accuracy:[0.991600000038147] 
epoch:360 ,loss:[0.017597671059668064], accuracy:[0.99492] 
epoch:361 ,loss:[0.01646732238829136], accuracy:[0.9952] 
epoch:362 ,loss:[0.020568441494852305], accuracy:[0.99374] 
epoch:363 ,loss:[0.08595641554117203], accuracy:[0.971999999961853] 
epoch:364 ,loss:[0.1452360127210617], accuracy:[0.952659999961853] 
epoch:365 ,loss:[0.10266850490748883], accuracy:[0.96524] 
epoch:366 ,loss:[0.05522408226132393], accuracy:[0.9811199999618531] 
epoch:367 ,loss:[0.03747712484002114], accuracy:[0.987860000038147] 
epoch:368 ,loss:[0

epoch:480 ,loss:[0.01190139924377203], accuracy:[0.99716] 
epoch:481 ,loss:[0.012938855390548705], accuracy:[0.9966800000190735] 
epoch:482 ,loss:[0.01696247554242611], accuracy:[0.99562] 
epoch:483 ,loss:[0.010500663404464722], accuracy:[0.9972600000190734] 
epoch:484 ,loss:[0.01006838237222284], accuracy:[0.99726] 
epoch:485 ,loss:[0.009266416100114585], accuracy:[0.99762] 
epoch:486 ,loss:[0.010248966519832611], accuracy:[0.9973] 
epoch:487 ,loss:[0.00686337735325098], accuracy:[0.9982] 
epoch:488 ,loss:[0.00984545982658863], accuracy:[0.99726] 
epoch:489 ,loss:[0.006879549319613725], accuracy:[0.99838] 
epoch:490 ,loss:[0.004286252047270536], accuracy:[0.99902] 
epoch:491 ,loss:[0.005222838683351875], accuracy:[0.99862] 
epoch:492 ,loss:[0.01990989954918623], accuracy:[0.9936600000190735] 
epoch:493 ,loss:[0.06529475567102433], accuracy:[0.977940000038147] 
epoch:494 ,loss:[0.07658495907872916], accuracy:[0.97616] 
epoch:495 ,loss:[0.04520008447945118], accuracy:[0.9852] 
epoch:496

MemoryError: 

In [10]:
    plt.plot(range(epoch_range), DNN2.loss)
    plt.plot(range(epoch_range), DNN2.accuracy)
    plt.plot(range(epoch_range), DNN2.grads)
    plt.show()


AttributeError: 'MY_DNN2' object has no attribute 'loss'